In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
feature_data = SETTING["feature_data"]

if feature_data is None:

    feature_data = {}

for data_name, data_dict in feature_data.items():

    print(data_name)

    data_dict["df"] = pd.read_table(data_dict["file_path"], index_col=0)

In [ ]:
sample_data = SETTING["sample_data"]

if sample_data is None:

    sample_data = {}

for data_name, data_dict in sample_data.items():

    print(data_name)

    data_dict["df"] = pd.read_table(data_dict["file_path"], index_col=0)

In [ ]:
w = pd.read_table(PATH["w.tsv"], index_col=0)

w.columns.name = "Factor"

h = pd.read_table(PATH["h.tsv"], index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
w__cluster_x_element = pd.read_table(PATH["w|cluster_x_column.tsv"], index_col=0)

w__cluster_x_element.columns.name = w.index.name

h__cluster_x_element = pd.read_table(PATH["h|cluster_x_column.tsv"], index_col=0)

h__cluster_x_element.columns.name = h.columns.name

In [ ]:
plot_only_columns_shared_by_target_and_all_data = True

output_directory_path = PATH["h|summary/"]

for target, target_type, data_dicts, directory_path, data_name_indices in (
    (
        h.loc["F2"],
        "continuous",
        sample_data,
        "{}/F2".format(PATH["h|match/"]),
        {"CTRP": ["oligomycin A", "spautin-1", "PF-750", "AZD7545"]},
    ),
    (
        h.loc["F3"],
        "continuous",
        sample_data,
        "{}/F3".format(PATH["h|match/"]),
        {
            "Binary Information": ["Head and Neck Cancer", "Squamous Cell Carcinoma"],
            "KRAS Factor": ["KRAS-AP1"],
            "Gene Set": ["AIGNER_ZEB1_TARGETS"],
            "Protein": ["EGFR"],
            "RNAi": ["EGFR", "ERBB2", "ERBB3"],
            "CRISPR": ["EGFR", "ERBB2", "ERBB3"],
            "NP24": ["Erlotinib", "Lapatinib", "AZD0530", "ZD-6474"],
            "CTRP": [
                "afatinib",
                "gefitinib",
                "erlotinib",
                "canertinib",
                "neratinib",
                "ibrutinib",
                "saracatinib",
                "dasatinib",
                "lapatinib",
            ],
        },
    ),
    (
        h.loc["F4"],
        "continuous",
        sample_data,
        "{}/F4".format(PATH["h|match/"]),
        {
            "KRAS Factor": ["EMT"],
            "RNAi": ["FGFR1", "PDGFRA"],
            "CRISPR": ["FGFR1", "PDGFRA"],
            "NP24": ["TKI258"],
            "CTRP": ["foretinib", "dasatinib", "imatinib", "cabozantinib", "axitinib"],
        },
    ),
):

    data_dicts_ = {}

    score_moe_p_value_fdr_dicts = {}

    for data_name, indices in data_name_indices.items():

        data_dicts_[data_name] = {
            "df": data_dicts[data_name]["df"].loc[indices],
            "type": data_dicts[data_name]["type"],
        }

        score_moe_p_value_fdr_dicts[data_name] = pd.read_table(
            "{}/{}.tsv".format(directory_path, ccal.make_file_name_from_str(data_name)),
            index_col=0,
        )

    title = "{} Summary".format(target.name)

    ccal.make_summary_match_panel(
        target,
        data_dicts_,
        score_moe_p_value_fdr_dicts,
        plot_only_columns_shared_by_target_and_all_data=plot_only_columns_shared_by_target_and_all_data,
        target_type=target_type,
        plot_std=SETTING["plot_std"],
        title=dict(text=title),
        xaxis_kwargs=dict(ticks="", showticklabels=False),
        html_file_path="{}/{}".format(
            output_directory_path, ccal.make_file_name_from_str(title)
        ),
    )